In [1]:
%matplotlib inline
import sys
sys.path.append("../")
sys.path.append('/home/hao/Research/probtorch/')
import numpy as np
import torch.nn as nn
import math
import matplotlib.pyplot as plt
from plots import *
from utils import *
from objectives import *
from torch.distributions.normal import Normal
from torch.distributions.one_hot_categorical import OneHotCategorical as cat
import time
import probtorch
print('probtorch:', probtorch.__version__, 
      'torch:', torch.__version__, 
      'cuda:', torch.cuda.is_available())

probtorch: 0.0+5a2c637 torch: 1.0.0 cuda: True


In [2]:
N = 300
K = 3
D = 2

## Model Parameters
SAMPLE_SIZE = 10
NUM_HIDDEN1 = 8
STAT_SIZE = 8
NUM_LATENTS =  D
## Training Parameters
BATCH_SIZE = 10
NUM_EPOCHS = 100
LEARNING_RATE = 1e-3
CUDA = torch.cuda.is_available()
PATH = 'gibbs-z-v4-varying-radius'

gpu = torch.device('cuda:1')

In [3]:
Xs = torch.from_numpy(np.load('rings_varying_radius/obs.npy')).float()
# STATES = torch.from_numpy(np.load('rings_dataset/states.npy')).float()
OBS_MU = torch.from_numpy(np.load('rings_varying_radius/obs_mu.npy')).float()
OBS_RAD = torch.from_numpy(np.load('rings_varying_radius/obs_rad.npy')).float()
NUM_SEQS = Xs.shape[0]
NUM_BATCHES = int((Xs.shape[0] / BATCH_SIZE))

In [4]:
class Enc_z(nn.Module):
    def __init__(self, num_hidden=8,
                       sample_size=SAMPLE_SIZE,
                       batch_size=BATCH_SIZE):
        super(self.__class__, self).__init__()
        self.log_prob = nn.Sequential(
            nn.Linear(2*D+2, D+2),
            nn.Tanh(),
            nn.Linear(D+2, D+2),
            nn.Tanh(),
            nn.Linear(D+2, 3),
            nn.Linear(3, 2),
            nn.Tanh(),
            nn.Linear(2, 2),
            nn.Tanh(),
            nn.Linear(2, 1))
        
        self.prior_pi = torch.ones(K) * (1./ K)
        self.noise_sigma = torch.ones((sample_size, batch_size, N, 1)) *  0.05
        if CUDA:
            self.prior_pi = self.prior_pi.cuda().to(gpu)
            self.noise_sigma = self.noise_sigma.cuda().to(gpu)
  
    def forward(self, obs, obs_mu, obs_rad, sample_size, batch_size, decay_factor):
        obs_mu_c1 = obs_mu[:, :, 0, :].unsqueeze(-2).repeat(1,1,N,1)
        obs_mu_c2 = obs_mu[:, :, 1, :].unsqueeze(-2).repeat(1,1,N,1)
        obs_mu_c3 = obs_mu[:, :, 2, :].unsqueeze(-2).repeat(1,1,N,1)
        
        obs_rad_c1 = obs_rad[:, :, 0, :].unsqueeze(-2).repeat(1,1,N,1)
        obs_rad_c2 = obs_rad[:, :, 1, :].unsqueeze(-2).repeat(1,1,N,1)
        obs_rad_c3 = obs_rad[:, :, 2, :].unsqueeze(-2).repeat(1,1,N,1)        
        # analytically compute L2 norm
        dist1 = ((obs - obs_mu_c1) ** 2).sum(-1).sqrt().unsqueeze(-1)
        dist2 = ((obs - obs_mu_c2) ** 2).sum(-1).sqrt().unsqueeze(-1)
        dist3 = ((obs - obs_mu_c3) ** 2).sum(-1).sqrt().unsqueeze(-1)
        
        pot1 = self.log_prob(torch.cat((-, obs_rad_c1, self.noise_sigma), -1))   
        pot2 = self.log_prob(torch.cat((obs, obs_mu_c2, obs_rad_c2, self.noise_sigma), -1))     
        pot3 = self.log_prob(torch.cat((obs, obs_mu_c2, obs_rad_c3, self.noise_sigma), -1))    

        pots = torch.cat((pot1, pot2, pot3), -1) # S * B * N * K


        q_pi = F.softmax(pots, -1)
        
        q = probtorch.Trace()
        p = probtorch.Trace()
        
        z = cat(q_pi).sample()
        _ = q.variable(cat, probs=q_pi, value=z, name='zs')
        _ = p.variable(cat, probs=self.prior_pi, value=z, name='zs')
        return q, p
    
def initialize():
    enc_z = Enc_z()
    if CUDA:
        enc_z.cuda().to(gpu)
    optimizer =  torch.optim.Adam(list(enc_z.parameters()),lr=LEARNING_RATE, betas=(0.9, 0.99))    
    return enc_z, optimizer

In [5]:
enc_z, optimizer = initialize()

In [ ]:
EUBOs = []
ELBOs = []
ESSs = []

flog = open('results/log-' + PATH + '.txt', 'w+')
flog.write('EUBO\tELBO\tESS\n')
flog.close()
for epoch in range(NUM_EPOCHS):
    time_start = time.time()
    indices = torch.randperm(NUM_SEQS)
    EUBO = 0.0
    ELBO = 0.0
    ESS = 0.0
    for step in range(NUM_BATCHES):
        optimizer.zero_grad()
        batch_indices = indices[step*BATCH_SIZE : (step+1)*BATCH_SIZE]
        obs = Xs[batch_indices]
        obs_mu = OBS_MU[batch_indices].repeat(SAMPLE_SIZE, 1, 1, 1)
        obs_rad = OBS_RAD[batch_indices].repeat(SAMPLE_SIZE, 1, 1, 1)
        obs = shuffler(obs).repeat(SAMPLE_SIZE, 1, 1, 1)
        if CUDA:
            obs = obs.cuda().to(gpu)
            obs_mu = obs_mu.cuda().to(gpu)
            obs_rad = obs_rad.cuda().to(gpu)
        eubo, elbo, ess = Eubo_gibbs_z(enc_z, obs, obs_mu, obs_rad, N, K, D, SAMPLE_SIZE, BATCH_SIZE, gpu, 0, noise_sigma=0.05)
        ## gradient step
        eubo.backward()
        optimizer.step()
        EUBO += eubo.item()
        ELBO += elbo.item()
        ESS += ess.item()
    EUBOs.append(EUBO / NUM_BATCHES)
    ELBOs.append(ELBO / NUM_BATCHES)
    ESSs.append(ESS / NUM_BATCHES) 
    flog = open('results/log-' + PATH + '.txt', 'a+')
    print('%.3f\t%.3f\t%.3f'
            % (EUBO/NUM_BATCHES, ELBO/NUM_BATCHES, ESS/NUM_BATCHES), file=flog)
    flog.close()
    time_end = time.time()
    print('epoch=%d, EUBO=%.3f, ELBO=%.3f, ESS=%.3f (%ds)'
            % (epoch, EUBO/NUM_BATCHES, ELBO/NUM_BATCHES, ESS/NUM_BATCHES, 
               time_end - time_start))

epoch=0, EUBO=-165.775, ELBO=-51089.733, ESS=3.868 (2s)
epoch=1, EUBO=-36.697, ELBO=-46699.274, ESS=4.028 (2s)
epoch=2, EUBO=-92.634, ELBO=-41478.734, ESS=4.270 (2s)
epoch=3, EUBO=-186.830, ELBO=-39753.794, ESS=4.374 (2s)
epoch=4, EUBO=-243.609, ELBO=-38489.520, ESS=4.449 (2s)
epoch=5, EUBO=-306.356, ELBO=-37030.957, ESS=4.538 (2s)
epoch=6, EUBO=-405.016, ELBO=-35711.824, ESS=4.641 (2s)
epoch=7, EUBO=-447.531, ELBO=-34717.156, ESS=4.749 (2s)
epoch=8, EUBO=-536.433, ELBO=-32663.978, ESS=4.951 (2s)
epoch=9, EUBO=-615.203, ELBO=-29582.488, ESS=5.374 (1s)
epoch=10, EUBO=-556.290, ELBO=-29780.627, ESS=5.584 (1s)
epoch=11, EUBO=-574.727, ELBO=-30379.248, ESS=5.663 (1s)
epoch=12, EUBO=-574.588, ELBO=-30926.358, ESS=5.688 (1s)
epoch=13, EUBO=-558.221, ELBO=-31303.861, ESS=5.696 (2s)
epoch=14, EUBO=-568.707, ELBO=-31562.124, ESS=5.696 (2s)
epoch=15, EUBO=-562.277, ELBO=-31607.651, ESS=5.710 (2s)
epoch=16, EUBO=-586.423, ELBO=-31707.540, ESS=5.720 (2s)
epoch=17, EUBO=-556.387, ELBO=-31716.253, E

KeyboardInterrupt: 

In [ ]:
BATCH_SIZE_TEST = 20

def sample_single_batch(num_seqs, Xs, OBS_MU, OBS_RAD, sample_size, batch_size, gpu):
    indices = torch.randperm(num_seqs)
    batch_indices = indices[0*batch_size : (0+1)*batch_size]
    obs = Xs[batch_indices]
    obs_mu = OBS_MU[batch_indices].repeat(sample_size, 1, 1, 1)
    obs_rad = OBS_RAD[batch_indices].repeat(sample_size, 1, 1, 1)
    obs = shuffler(obs).repeat(sample_size, 1, 1, 1)
    if CUDA:
        obs = obs.cuda().to(gpu)
        obs_mu = obs_mu.cuda().to(gpu)
        obs_rad = obs_rad.cuda().to(gpu)
    return obs, obs_mu, obs_rad

def test(enc_z, obs, obs_mu, obs_rad, sample_size, batch_size):
        ## update z -- cluster assignments
    q_z, p_z = enc_z(obs, obs_mu, obs_rad, sample_size, batch_size, 0)
    return q_z

def plot_samples(obs, q_eta, q_z, K, batch_size, PATH):
    colors = ['r', 'b', 'g']
    fig = plt.figure(figsize=(25,20))
    xs = obs[0].cpu().data.numpy()
    mu = obs_mu[0].cpu().data.numpy()
    zs = q_z['zs'].dist.probs[0].cpu().data.numpy()
    for b in range(batch_size):
        ax = fig.add_subplot(int(batch_size / 5), 5, b+1)
        x = xs[b]
        z = zs[b]
        mu_b = mu[b]
        assignments = z.argmax(-1)
        for k in range(K):
            xk = x[np.where(assignments == k)]
            ax.scatter(xk[:, 0], xk[:, 1], c=colors[k], alpha=0.2)
            ax.scatter(mu_b[k, 0], mu_b[k, 1], c=colors[k])
        ax.set_ylim([-8, 8])
        ax.set_xlim([-8, 8])
    plt.savefig('results/modes-' + PATH + '.svg')
    
obs, obs_mu, obs_rad = sample_single_batch(NUM_SEQS, Xs, OBS_MU, OBS_RAD, SAMPLE_SIZE, BATCH_SIZE_TEST, gpu)
q_z = test(enc_z, obs, obs_mu, obs_rad, SAMPLE_SIZE, BATCH_SIZE_TEST)
%time plot_samples(obs, obs_mu, q_z, K, BATCH_SIZE_TEST, PATH)